In [2]:
## Imports

import pandas as pd
import io
import matplotlib.pyplot as plt
import time
from sklearn.metrics import roc_auc_score

import re
import string
import nltk
import itertools
import numpy as np

# this allows you to apply a progress bar to pandas operations
from tqdm import tqdm
tqdm.pandas()

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import pandas as pd
words_scores = [
               [["among us yo", "garbage dfsd", "kdhalq"], [0.8, 0.2, 0.1]],
               [["among us", "garbage dfsd", "kdhalq"], [0.8, 0.2, 0.1]],
               [["amongus"],[1.0]],
               [['innerslothpresent', 'amongus'], [1, 1]]
                ]
test_df = pd.DataFrame()
test_df["words_scores"] = words_scores
test_df

,words_scores
0,"[[among us yo, garbage dfsd, kdhalq], [0.8, 0...."
1,"[[among us, garbage dfsd, kdhalq], [0.8, 0.2, ..."
2,"[[amongus], [1.0]]"
3,"[[innerslothpresent, amongus], [1, 1]]"


In [4]:
def new_score_text(row):
    sample_list = row["words_scores"][0]
    score_list = row["words_scores"][1]
    new_text = []
    new_score = []
    for text, score in zip(sample_list, score_list):
        # print("text: ",text)
        space_number = text.count(" ")
        # print("number of words: ",word_number)
        for i in range(space_number+1):
            new_score.append(score)
        new_text.extend(text.split())
        new_text_score= [new_text, new_score]
    return new_text_score

In [5]:
test_df["new_word_scores"] = test_df.apply(new_score_text, axis=1)
test_df
# test_df.drop(["words_scores"], axis=1)

,words_scores,new_word_scores
0,"[[among us yo, garbage dfsd, kdhalq], [0.8, 0....","[[among, us, yo, garbage, dfsd, kdhalq], [0.8,..."
1,"[[among us, garbage dfsd, kdhalq], [0.8, 0.2, ...","[[among, us, garbage, dfsd, kdhalq], [0.8, 0.8..."
2,"[[amongus], [1.0]]","[[amongus], [1.0]]"
3,"[[innerslothpresent, amongus], [1, 1]]","[[innerslothpresent, amongus], [1, 1]]"


In [6]:
targets = ["among us", "imposter"]

In [7]:
# helper functions for funal scoring function

def advance(iterator, step):
    next(itertools.islice(iterator, step, step), None)

def tuplewize(iterable, size):
    iterators = itertools.tee(iterable, size)
    for position, iterator in enumerate(iterators):
        advance(iterator, position)
    return zip(*iterators)

In [8]:
# define function to produce final scores

def get_final_score(row):
    sample_list = row["new_word_scores"][0]
    score_list = row["new_word_scores"][1]
    max_score = 0
    
    for target in targets:
        print("target: ", target)
        phrase_length = len(target.split())
        print("phrase length: ", phrase_length)
        for i in range(1, phrase_length+1):
            print("i:", i)
            for c, s in zip(tuplewize(sample_list, i), tuplewize(score_list, i)):
                print("\n c: ",c)
                print("s: ",s)
                new_sample = " ".join(c)
                print("new_sample: ", new_sample)
                edit_distance = nltk.edit_distance(target, new_sample)
                print("edit_distance: ", edit_distance)
                closeness = (len(target)-edit_distance)/len(target)
                print("closeness: ", closeness)
                confidence_score = np.mean(s)
                print("confidence_score: ", confidence_score)
                final_score = closeness * confidence_score
                print("final_score: ", final_score)
                if final_score > max_score:
                    max_score = final_score

        print("max_score: ", max_score) 
        return max_score

test_df["OCR_score"] = test_df.apply(get_final_score, axis=1)
test_df

target:  among us
phrase length:  2
i: 1

 c:  ('among',)
s:  (0.8,)
new_sample:  among
edit_distance:  3
closeness:  0.625
confidence_score:  0.8
final_score:  0.5

 c:  ('us',)
s:  (0.8,)
new_sample:  us
edit_distance:  6
closeness:  0.25
confidence_score:  0.8
final_score:  0.2

 c:  ('yo',)
s:  (0.8,)
new_sample:  yo
edit_distance:  7
closeness:  0.125
confidence_score:  0.8
final_score:  0.1

 c:  ('garbage',)
s:  (0.2,)
new_sample:  garbage
edit_distance:  7
closeness:  0.125
confidence_score:  0.2
final_score:  0.025

 c:  ('dfsd',)
s:  (0.2,)
new_sample:  dfsd
edit_distance:  8
closeness:  0.0
confidence_score:  0.2
final_score:  0.0

 c:  ('kdhalq',)
s:  (0.1,)
new_sample:  kdhalq
edit_distance:  8
closeness:  0.0
confidence_score:  0.1
final_score:  0.0
i: 2

 c:  ('among', 'us')
s:  (0.8, 0.8)
new_sample:  among us
edit_distance:  0
closeness:  1.0
confidence_score:  0.8
final_score:  0.8

 c:  ('us', 'yo')
s:  (0.8, 0.8)
new_sample:  us yo
edit_distance:  7
closeness:  0.12

,words_scores,new_word_scores,OCR_score
0,"[[among us yo, garbage dfsd, kdhalq], [0.8, 0....","[[among, us, yo, garbage, dfsd, kdhalq], [0.8,...",0.800
1,"[[among us, garbage dfsd, kdhalq], [0.8, 0.2, ...","[[among, us, garbage, dfsd, kdhalq], [0.8, 0.8...",0.800
2,"[[amongus], [1.0]]","[[amongus], [1.0]]",0.875
3,"[[innerslothpresent, amongus], [1, 1]]","[[innerslothpresent, amongus], [1, 1]]",0.875
